In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
# URL to scrape
url = 'https://rivalsmeta.com/player/29526699'

In [3]:
# --- Configure Selenium ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # run headless if desired
driver = webdriver.Chrome(options=chrome_options)

In [4]:
# # --- Configure Selenium ---
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # run headless if desired

# # Specify the full path to chromedriver.exe
# chromedriver_path = r'C:\Users\bmac9\Desktop\Coding Stuff\chromedriver-win32\chromedriver.exe'
# service = Service(executable_path=chromedriver_path)
# driver = webdriver.Chrome(service=service, options=chrome_options)

In [5]:
# ----------------------------------------
# Timer: Start
# ----------------------------------------
start_time = time.time()

# ----------------------------------------
# STEP 0: Get the most recent Replay ID from the SQLite database (marvel.db)
# ----------------------------------------
print('---------- Step 0 ----------')
db_filename = 'marvel.db'
table_name = 'matches'
last_replay_id = None

try:
    conn = sqlite3.connect(db_filename)
    existing_df = pd.read_sql(f"SELECT [Replay ID] FROM {table_name} ORDER BY game_id DESC LIMIT 1", conn)
    if not existing_df.empty:
        last_replay_id = existing_df["Replay ID"].iloc[0]
        print("Most recent scraped Replay ID from DB:", last_replay_id)
    else:
        print("No matches found in the DB.")
    conn.close()
except Exception as e:
    print("Could not read from DB (table may not exist). Starting fresh.")
    last_replay_id = None

# ----------------------------------------
# STEP 1: Set up Selenium and load the page
# ----------------------------------------
print('\n---------- Step 1 ----------')
# --- Configure Selenium ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # run headless if desired
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)  # wait for initial content to load

# Define CSS selectors for match history and button
history_container_selector = "div.match-history.left-comp"
show_more_button_selector = ".show-more-btn"
match_details_selector = "div.match-details"

# ----------------------------------------
# STEP 2: Click "Show More" until no new matches are loaded.
# ----------------------------------------
print('\n---------- Step 2 ----------')
prev_count = len(driver.find_elements(By.CSS_SELECTOR, match_details_selector))
print("Initial match count:", prev_count)

while True:
    try:
        history_container = driver.find_element(By.CSS_SELECTOR, history_container_selector)
        show_more_button = history_container.find_element(By.CSS_SELECTOR, show_more_button_selector)
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
        show_more_button.click()
        time.sleep(2)  # wait for new content to load

        current_count = len(driver.find_elements(By.CSS_SELECTOR, match_details_selector))
        print("Match count after click:", current_count)
        if current_count == prev_count:
            print("No new matches loaded. Stopping 'Show More' clicks.")
            break
        else:
            prev_count = current_count
    except Exception as e:
        print("Error clicking 'Show More':", e)
        break

# ----------------------------------------
# STEP 3: Process each match one by one:
#         For each match, click its dropdown trigger to reveal its Replay ID,
#         print the Replay ID, and stop processing further if it equals last_replay_id.
# ----------------------------------------
print('\n---------- Step 3 ----------')
matches = driver.find_elements(By.CSS_SELECTOR, match_details_selector)
print("Found", len(matches), "match elements.")

new_matches_data = []  # To store data for new matches

for idx, match in enumerate(matches, start=1):
    try:
        # Click the dropdown trigger inside this match.
        triggers = match.find_elements(By.CSS_SELECTOR, "div.link-ind")
        if triggers:
            trigger = triggers[0]
            driver.execute_script("arguments[0].scrollIntoView(true);", trigger)
            driver.execute_script("arguments[0].click();", trigger)
            time.sleep(0.5)  # allow dropdown to expand
        else:
            print(f"Match {idx}: No dropdown trigger found.")
        
        # Attempt to locate the dropdown using a relative XPath.
        dropdowns = match.find_elements(By.XPATH, "./following-sibling::div[contains(@class, 'match-dropdown')]")
        if not dropdowns:
            # Fallback: search within the match element itself.
            dropdowns = match.find_elements(By.CSS_SELECTOR, "div.match-dropdown")
        if dropdowns:
            dropdown = dropdowns[0]
            # If the Replay ID is not visible, click the dropdown to ensure it's expanded.
            replay_id_elems = dropdown.find_elements(By.CSS_SELECTOR, "div.replay-id span")
            if not replay_id_elems or replay_id_elems[0].text.strip() == "":
                driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
                driver.execute_script("arguments[0].click();", dropdown)
                time.sleep(0.5)
                replay_id_elems = dropdown.find_elements(By.CSS_SELECTOR, "div.replay-id span")
            replay_id = replay_id_elems[0].text.strip() if replay_id_elems else ""
            print(f"Match {idx} Replay ID: {replay_id}")
            
            # If this match's Replay ID equals the stored one, stop processing further.
            if last_replay_id and replay_id == last_replay_id:
                print(f"Encountered already scraped Replay ID: {replay_id}. Stopping further processing.")
                break
            
            # (Optional) Store the replay ID and index.
            new_matches_data.append({"match_index": idx, "Replay ID": replay_id})
        else:
            print(f"Match {idx}: No dropdown found.")
    except Exception as e:
        print(f"Error processing match {idx}: {e}")
        
# Capture the full HTML.
html = driver.page_source

# Quit the driver.
driver.quit()

# ----------------------------------------
# STEP 4: Build a full match table from the page using BeautifulSoup.
#         This is an alternative approach if you wish to extract all data from the HTML.
# ----------------------------------------
print('\n---------- Step 4 ----------')
soup = BeautifulSoup(html, 'html.parser')
match_details_list = soup.find_all("div", class_="match-details")
print("Total match details found (BeautifulSoup):", len(match_details_list))

match_table = []
total_games = len(match_details_list)
for i, match in enumerate(match_details_list):
    game_id = total_games - i  # most recent gets highest number
    
    # Extract map and score.
    map_section = match.find("div", class_="map")
    if map_section:
        map_name = map_section.find("div", class_="name").get_text(strip=True) if map_section.find("div", class_="name") else ""
        score_container = map_section.find("div", class_="image")
        if score_container:
            score_div = score_container.find("div", class_="score")
            score = ":".join([span.get_text(strip=True) for span in score_div.find_all("span")]) if score_div else ""
        else:
            score = ""
    else:
        map_name = ""
        score = ""
    
    # Extract match length and result.
    hero_section = match.find("div", class_="hero")
    if hero_section:
        hero_wrapper = hero_section.find("div", class_="wrapper")
        if hero_wrapper:
            spans = hero_wrapper.find("div", class_="duration").find_all("span") if hero_wrapper.find("div", class_="duration") else []
            match_length = spans[1].get_text(strip=True) if len(spans) > 1 else ""
            result_text = spans[0].get_text(strip=True) if spans else ""
        else:
            match_length = ""
            result_text = ""
    else:
        match_length = ""
        result_text = ""
    
    # Extract game mode.
    match_link = match.find("a", class_="match")
    if match_link:
        stats_section = match_link.find("div", class_="stats")
        stat_div = stats_section.find("div", class_="stat") if stats_section else None
        game_mode = stat_div.find("div", class_="value").get_text(strip=True) if stat_div else ""
    else:
        game_mode = ""
    
    # Extract replay ID.
    dropdown = match.find_next("div", class_="match-dropdown")
    if dropdown:
        replay_id_container = dropdown.find("div", class_="replay-id")
        replay_id = replay_id_container.find("span").get_text(strip=True) if replay_id_container and replay_id_container.find("span") else ""
    else:
        replay_id = ""
    
    if last_replay_id and replay_id == last_replay_id:
        print("Encountered already scraped Replay ID:", replay_id)
        print("Stopping processing of further matches in BeautifulSoup parsing.")
        break
    
    match_table.append({
        "game_id": game_id,
        "Map": map_name,
        "Score": score,
        "Replay ID": replay_id,
        "Match Length": match_length,
        "Result": result_text,
        "Game Mode": game_mode
    })

df_match_table = pd.DataFrame(match_table)

# ----------------------------------------
# STEP 5: Append new matches to the SQLite database (marvel.db)
# ----------------------------------------
print('\n---------- Step 5 ----------')
if not df_match_table.empty:
    conn = sqlite3.connect(db_filename)
    df_match_table.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()
    print("New matches appended to the database.")
else:
    print("No new matches found to append.")

# ----------------------------------------
# Timer: End and print elapsed time.
# ----------------------------------------
end_time = time.time()
elapsed = end_time - start_time
print(f"\nElapsed time: {elapsed:.2f} seconds")

print('\n---------- New Data ----------')

print("\nScraped Match Table (BeautifulSoup):")
df_match_table

---------- Step 0 ----------
Could not read from DB (table may not exist). Starting fresh.

---------- Step 1 ----------

---------- Step 2 ----------
Initial match count: 20
Match count after click: 40
Match count after click: 60
Match count after click: 80
Match count after click: 100
Match count after click: 120
Match count after click: 140
Match count after click: 160
Match count after click: 169
Match count after click: 169
No new matches loaded. Stopping 'Show More' clicks.

---------- Step 3 ----------
Found 169 match elements.
Match 1 Replay ID: 10461226090
Match 2 Replay ID: 10572222444
Match 3 Replay ID: 10506427193
Match 4 Replay ID: 10976523234
Match 5 Replay ID: 10622326105
Match 6 Replay ID: 10622618928
Match 7 Replay ID: 10359312779
Match 8 Replay ID: 10444127481
Match 9 Replay ID: 10356417523
Match 10 Replay ID: 10198425631
Match 11 Replay ID: 10715927907
Match 12 Replay ID: 10968131015
Match 13 Replay ID: 10407924082
Match 14 Replay ID: 10370823436
Match 15 Replay ID: 

,game_id,Map,Score,Replay ID,Match Length,Result,Game Mode
0,169,YGGDRASILL PATH,2:3,10461226090,14m 52s,LOSS,Competitive
1,168,ROYAL PALACE,1:2,10572222444,14m 53s,LOSS,Competitive
2,167,SYMBIOTIC SURFACE,1:2,10506427193,8m 0s,LOSS,Competitive
3,166,MIDTOWN,3:4,10976523234,19m 13s,LOSS,Competitive
4,165,HALL OF DJALIA,0:1,10622326105,6m 14s,LOSS,Competitive
...,...,...,...,...,...,...,...
164,5,HALL OF DJALIA,0:3,10217714075,8m 43s,LOSS,Competitive
165,4,SYMBIOTIC SURFACE,0:1,10948919995,8m 0s,LOSS,Competitive
166,3,MIDTOWN,,1073369220,6m 47s,WIN,Quick Play
167,2,HELL'S HEAVEN,1:2,10973013613,9m 59s,LOSS,Quick Play


In [6]:
# Connect to the SQLite database
conn = sqlite3.connect('marvel.db')

# Verify the data
print("Matches table:")
print(pd.read_sql('SELECT * FROM matches ORDER BY game_id DESC;', conn))

# Close the connection when done
conn.close()

Matches table:
     game_id                Map Score    Replay ID Match Length Result  \
0        169    YGGDRASILL PATH   2:3  10461226090      14m 52s   LOSS   
1        168       ROYAL PALACE   1:2  10572222444      14m 53s   LOSS   
2        167  SYMBIOTIC SURFACE   1:2  10506427193        8m 0s   LOSS   
3        166            MIDTOWN   3:4  10976523234      19m 13s   LOSS   
4        165     HALL OF DJALIA   0:1  10622326105       6m 14s   LOSS   
..       ...                ...   ...          ...          ...    ...   
164        5     HALL OF DJALIA   0:3  10217714075       8m 43s   LOSS   
165        4  SYMBIOTIC SURFACE   0:1  10948919995        8m 0s   LOSS   
166        3            MIDTOWN         1073369220       6m 47s    WIN   
167        2      HELL'S HEAVEN   1:2  10973013613       9m 59s   LOSS   
168        1    YGGDRASILL PATH        10148911466       11m 4s   LOSS   

       Game Mode  
0    Competitive  
1    Competitive  
2    Competitive  
3    Competitive  
4

In [7]:
# ----------------------------------------
# Timer: Start
# ----------------------------------------
start_time = time.time()

# ----------------------------------------
# STEP 0: Get the most recent Replay ID from the SQLite database (marvel.db)
# ----------------------------------------
print('---------- Step 0 ----------')
db_filename = 'marvel.db'
matches_table = 'matches'
stats_table = 'stats'
last_replay_id = None

# Create a connection to the database.
conn = sqlite3.connect(db_filename)

# Use a SQL join to combine the matches and dropdown_stats tables on game_id.
# Then order by the game_id descending and limit the result to 1.
sql = f"""
SELECT m.[Replay ID]
FROM {matches_table} m
JOIN {stats_table} d ON m.game_id = d.game_id
ORDER BY m.game_id DESC
LIMIT 1;
"""

try:
    existing_df = pd.read_sql(sql, conn)
    if not existing_df.empty:
        last_replay_id = existing_df["Replay ID"].iloc[0]
        print("Most recent scraped Replay ID from joined tables:", last_replay_id)
    else:
        print("No matches found in the joined tables.")
except Exception as e:
    print("Error reading from the database:", e)
finally:
    conn.close()

# ----------------------------------------
# STEP 1: Set up Selenium and load the page
# ----------------------------------------
print('\n---------- Step 1 ----------')
# --- Configure Selenium ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # run headless if desired
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)  # wait for initial content to load

# Define CSS selectors for match history and button
history_container_selector = "div.match-history.left-comp"
show_more_button_selector = ".show-more-btn"
match_details_selector = "div.match-details"

# ----------------------------------------
# STEP 2: Click "Show More" until no new matches are loaded.
# ----------------------------------------
print('\n---------- Step 2 ----------')
prev_count = len(driver.find_elements(By.CSS_SELECTOR, match_details_selector))
print("Initial match count:", prev_count)

while True:
    try:
        history_container = driver.find_element(By.CSS_SELECTOR, history_container_selector)
        show_more_button = history_container.find_element(By.CSS_SELECTOR, show_more_button_selector)
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
        show_more_button.click()
        time.sleep(2)  # wait for new content to load

        current_count = len(driver.find_elements(By.CSS_SELECTOR, match_details_selector))
        print("Match count after click:", current_count)
        if current_count == prev_count:
            print("No new matches loaded. Stopping 'Show More' clicks.")
            break
        else:
            prev_count = current_count
    except Exception as e:
        print("Error clicking 'Show More':", e)
        break

total_matches = current_count

# ----------------------------------------
# STEP 3: Process each match one by one:
#         For each match, click its dropdown trigger to reveal its Replay ID,
#         print the Replay ID, and stop processing further if it equals last_replay_id.
# ----------------------------------------
print('\n---------- Step 3 ----------')
matches = driver.find_elements(By.CSS_SELECTOR, match_details_selector)
print("Found", len(matches), "match elements.")

new_matches_data = []  # To store data for new matches

for idx, match in enumerate(matches, start=1):
    try:
        # Click the dropdown trigger inside this match.
        triggers = match.find_elements(By.CSS_SELECTOR, "div.link-ind")
        if triggers:
            trigger = triggers[0]
            driver.execute_script("arguments[0].scrollIntoView(true);", trigger)
            driver.execute_script("arguments[0].click();", trigger)
            time.sleep(0.5)  # allow dropdown to expand
        else:
            print(f"Match {idx}: No dropdown trigger found.")
        
        # Attempt to locate the dropdown using a relative XPath.
        dropdowns = match.find_elements(By.XPATH, "./following-sibling::div[contains(@class, 'match-dropdown')]")
        if not dropdowns:
            # Fallback: search within the match element itself.
            dropdowns = match.find_elements(By.CSS_SELECTOR, "div.match-dropdown")
        if dropdowns:
            dropdown = dropdowns[0]
            # If the Replay ID is not visible, click the dropdown to ensure it's expanded.
            replay_id_elems = dropdown.find_elements(By.CSS_SELECTOR, "div.replay-id span")
            if not replay_id_elems or replay_id_elems[0].text.strip() == "":
                driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
                driver.execute_script("arguments[0].click();", dropdown)
                time.sleep(0.5)
                replay_id_elems = dropdown.find_elements(By.CSS_SELECTOR, "div.replay-id span")
            replay_id = replay_id_elems[0].text.strip() if replay_id_elems else ""
            print(f"Match {idx} Replay ID: {replay_id}")
            
            # If this match's Replay ID equals the stored one, stop processing further.
            if last_replay_id and replay_id == last_replay_id:
                print(f"Encountered already scraped Replay ID: {replay_id}. Stopping further processing.")
                break
            
            # (Optional) Store the replay ID and index.
            new_matches_data.append({"match_index": idx, "Replay ID": replay_id})
        else:
            print(f"Match {idx}: No dropdown found.")
    except Exception as e:
        print(f"Error processing match {idx}: {e}")

# Capture the full HTML.
html = driver.page_source
        
# Quit the driver.
driver.quit()

# ----------------------------------------
# STEP 4: Parse the HTML and build the drop down stats table.
#         Process each dropdown in order.
#         Stop processing further drop-downs if a Replay ID equals last_replay_id.
# ----------------------------------------
print('\n---------- Step 4 ----------')
soup = BeautifulSoup(html, 'html.parser')
match_dropdowns = soup.find_all("div", class_="match-dropdown")
print("Found", len(match_dropdowns), "match dropdowns.")

dropdown_stats = []  # List to store stats from each dropdown

for idx, dropdown in enumerate(match_dropdowns, start=1):
    
    # Compute game_id so that the most recent game gets the highest number.
    game_id = total_matches - idx + 1
    
    # First, try to extract the Replay ID from this dropdown.
    replay_id_container = dropdown.find("div", class_="replay-id")
    replay_id = ""
    if replay_id_container:
        replay_span = replay_id_container.find("span")
        replay_id = replay_span.get_text(strip=True) if replay_span else ""
    print(f"Dropdown {idx} (game_id {game_id}) Replay ID: {replay_id}")
    
    # If this dropdown's Replay ID equals the stored one, stop processing further.
    if last_replay_id and replay_id == last_replay_id:
        print(f"Encountered already scraped Replay ID: {replay_id}. Stopping further processing of dropdowns.")
        break

    # Otherwise, process the drop-down stats table.
    table = dropdown.find("table")
    if not table:
        continue
    # Optionally, extract headers if needed (here we assume a fixed header order).
    # Extract data rows (skip header row)
    for row in table.find_all("tr")[1:]:
        cells = row.find_all("td")
        if not cells or len(cells) < 6:
            continue
        
        # Extract player name from the first cell.
        player_cell = cells[0]
        name_div = player_cell.find("div", class_="name")
        player_name = name_div.get_text(strip=True) if name_div else player_cell.get_text(strip=True)
        
        # Extract the other cells.
        rank = cells[1].get_text(strip=True)
        avg_div = cells[2].find("div", class_="avg")
        if avg_div:
            kda = avg_div.get_text(strip=True)
        else:
            kda = cells[2].get_text(strip=True)
        damage = cells[3].get_text(strip=True)
        dmg_taken = cells[4].get_text(strip=True)
        healing = cells[5].get_text(strip=True)
        
        # --- Extract Heroes Played and Time Played ---
        # First, try to find an "other-heroes" container within the current row
        other_heroes = row.find("div", class_="other-heroes")
        if not other_heroes:
            # If not found in the row, fall back to the parent match-details
            parent_match = dropdown.find_previous("div", class_="match-details")
            if parent_match:
                other_heroes = parent_match.find("div", class_="other-heroes")
        
        # Create a mapping from hero IDs to common names.
        hero_mapping = {
            "1011001": "Hulk",
            "1014001": "Punisher",
            "1015001": "Storm",
            "1016001": "Loki",
            "1018001": "Dr. Strange",
            "1020001": "Mantis",
            "1021001": "Hawkeye",
            "1022001": "Captain America",
            "1023001": "Rocket",
            "1024001": "Hela",
            "1025001": "Cloak and Dagger",
            "1026001": "Black Panther",
            "1027001": "Groot",
            "1029001": "Majic",
            "1030001": "Moon Knight",
            "1031001": "Luna Snow",
            "1032001": "Squirrel Girl",
            "1033001": "Black Widow",
            "1034001": "Iron Man",
            "1035001": "Venom",
            "1036001": "Spiderman",
            "1037001": "Magneto",
            "1038001": "Scarlet Witch",
            "1039001": "Thor",
            "1040001": "Mr. Fantastic",
            "1041001": "Bucky",
            "1042001": "Penny",
            "1043001": "Star Lord",
            "1045001": "Namor",
            "1046001": "Adam Warlocke",
            "1047001": "Jeff",
            "1048001": "Psylocke",
            "1049001": "Wolverine",
            "1050001": "Invisible Woman",
            "1052001": "Iron Fist",
        }

        hero_names = []
        times = []
        if other_heroes:
            hero_divs = other_heroes.find_all("div", class_="hero")
            for hero in hero_divs:
                img = hero.find("img")
                if img:
                    src = img.get("src", "")
                    # Attempt to extract an identifier from the src (e.g. "img_selecthero_1042001.png")
                    parts = src.split("img_selecthero_")
                    if len(parts) > 1:
                        hero_id = parts[1].split(".")[0]
                        # Use the mapping to get the common name if it exists, else fall back to the ID
                        common_name = hero_mapping.get(hero_id, hero_id)
                        hero_names.append(common_name)
                    else:
                        hero_names.append(src)
                else:
                    hero_names.append("N/A")
                time_div = hero.find("div", class_="time")
                if time_div:
                    times.append(time_div.get_text(strip=True))
                else:
                    times.append("N/A")
            heroes_played_str = " / ".join(hero_names)
            time_played_str = " / ".join(times)
        else:
            heroes_played_str = ""
            time_played_str = ""
        
        # Build a dictionary with the extracted data, including the new columns
        dropdown_stats.append({
            "game_id": game_id,
            "Player": player_name,
            "Rank": rank,
            "KDA": kda,
            "Damage": damage,
            "Dmg Taken": dmg_taken,
            "Healing": healing,
            "Heroes Played": heroes_played_str,
            "Time Played": time_played_str
        })

# Convert the list of dictionaries into a DataFrame
df_dropdown_stats = pd.DataFrame(dropdown_stats)

# ----------------------------------------
# OPTIONAL: Remove duplicates by checking if the stats already exist in the database.
# ----------------------------------------
print('\n---------- Duplicate Check ----------')
conn = sqlite3.connect(db_filename)
try:
    # Read existing stats data (excluding game_id, as it may differ)
    existing_stats_df = pd.read_sql(
        "SELECT Player, Rank, KDA, Damage, \"Dmg Taken\", Healing, \"Heroes Played\", \"Time Played\" FROM stats",
        conn
    )
except Exception as e:
    print("Error reading stats from DB, assuming none exist:", e)
    existing_stats_df = pd.DataFrame()
conn.close()

columns_to_check = ["Player", "Rank", "KDA", "Damage", "Dmg Taken", "Healing", "Heroes Played", "Time Played"]

def is_duplicate(row, existing_df):
    if existing_df.empty:
        return False
    # Create a boolean mask where all columns match.
    mask = (existing_df[columns_to_check] == row[columns_to_check]).all(axis=1)
    return mask.any()

# Filter out duplicate rows
df_dropdown_stats_unique = df_dropdown_stats[~df_dropdown_stats.apply(lambda row: is_duplicate(row, existing_stats_df), axis=1)]
print("After duplicate check, new stats records to append:")
print(df_dropdown_stats_unique)


# ----------------------------------------
# STEP 5: Append new matches to the SQLite database (marvel.db)
# ----------------------------------------
print('\n---------- Step 5 ----------')
if not df_dropdown_stats_unique.empty:
    conn = sqlite3.connect(db_filename)
    df_dropdown_stats_unique.to_sql(stats_table, conn, if_exists='append', index=False)
    conn.close()
    print("New matches appended to the database.")
else:
    print("No new matches found to append.")

# ----------------------------------------
# Timer: End and print elapsed time.
# ----------------------------------------
end_time = time.time()
elapsed = end_time - start_time
print(f"\nElapsed time: {elapsed:.2f} seconds")

print('\n---------- New Data ----------')

print("Dropdown stats DataFrame with heroes and times:")
df_dropdown_stats_unique

---------- Step 0 ----------
No matches found in the joined tables.

---------- Step 1 ----------

---------- Step 2 ----------
Initial match count: 20
Match count after click: 40
Match count after click: 60
Match count after click: 80
Match count after click: 100
Match count after click: 120
Match count after click: 140
Match count after click: 160
Match count after click: 169
Match count after click: 169
No new matches loaded. Stopping 'Show More' clicks.

---------- Step 3 ----------
Found 169 match elements.
Match 1 Replay ID: 10461226090
Match 2 Replay ID: 10572222444
Match 3 Replay ID: 10506427193
Match 4 Replay ID: 10976523234
Match 5 Replay ID: 10622326105
Match 6 Replay ID: 10622618928
Match 7 Replay ID: 10359312779
Match 8 Replay ID: 10444127481
Match 9 Replay ID: 10356417523
Match 10 Replay ID: 10198425631
Match 11 Replay ID: 10715927907
Match 12 Replay ID: 10968131015
Match 13 Replay ID: 10407924082
Match 14 Replay ID: 10370823436
Match 15 Replay ID: 10118218896
Match 16 Re

Dropdown 84 (game_id 86) Replay ID: 10967628622
Dropdown 85 (game_id 85) Replay ID: 10568122976
Dropdown 86 (game_id 84) Replay ID: 10272324130
Dropdown 87 (game_id 83) Replay ID: 10558318020
Dropdown 88 (game_id 82) Replay ID: 10728314955
Dropdown 89 (game_id 81) Replay ID: 10718520440
Dropdown 90 (game_id 80) Replay ID: 10262712225
Dropdown 91 (game_id 79) Replay ID: 10889222478
Dropdown 92 (game_id 78) Replay ID: 10123910381
Dropdown 93 (game_id 77) Replay ID: 10840216972
Dropdown 94 (game_id 76) Replay ID: 10741911494
Dropdown 95 (game_id 75) Replay ID: 10509719461
Dropdown 96 (game_id 74) Replay ID: 10179319141
Dropdown 97 (game_id 73) Replay ID: 10120612187
Dropdown 98 (game_id 72) Replay ID: 10403020358
Dropdown 99 (game_id 71) Replay ID: 10669417973
Dropdown 100 (game_id 70) Replay ID: 10938921867
Dropdown 101 (game_id 69) Replay ID: 10758523967
Dropdown 102 (game_id 68) Replay ID: 10116816801
Dropdown 103 (game_id 67) Replay ID: 10937518866
Dropdown 104 (game_id 66) Replay ID:

,game_id,Player,Rank,KDA,Damage,Dmg Taken,Healing,Heroes Played,Time Played
0,169,trick jokester,Gold 1,22 /13/ 0,"24,859","8,875",0,Squirrel Girl / Punisher / Moon Knight / Iron Man,7m / 3m / 3m / 45s
1,169,.-.',Platinum 3,22 /20/ 3,"22,769","13,591",0,Bucky / Namor / Hela / Psylocke,11m / 2m / 32s / 0s
2,169,s' more,Gold 1,1 /12/ 20,"2,294","8,252","20,904",Rocket / Luna Snow,10m / 4m
3,169,6macb,Gold 1,15 /8/ 13,"8,508","9,298","29,882",Cloak and Dagger,14m
4,169,TravyPG,Gold 1,14 /11/ 1,"17,027","45,415",0,Magneto / Dr. Strange,8m / 6m
...,...,...,...,...,...,...,...,...,...
2018,1,Lephsh,,15 /12/ 6,"12,987","4,750",0,Storm / Hawkeye,11m / 0s
2019,1,Paxton5Star,,12 /9/ 7,"6,046","10,136","12,797",Invisible Woman,11m
2020,1,ClassyRoboto,,10 /13/ 0,"7,388","12,817",0,Mr. Fantastic,11m
2021,1,TheAskaLottle,,17 /10/ 0,"8,140","16,907",0,Thor,11m


In [8]:
# Connect to the SQLite database
conn = sqlite3.connect('marvel.db')

# Verify the data
print("Stats table:")
print(pd.read_sql('SELECT * FROM stats ORDER BY game_id DESC;', conn))

# Close the connection when done
conn.close()

Stats table:
      game_id          Player        Rank        KDA  Damage Dmg Taken  \
0         169  trick jokester      Gold 1  22 /13/ 0  24,859     8,875   
1         169            .-.'  Platinum 3  22 /20/ 3  22,769    13,591   
2         169         s' more      Gold 1  1 /12/ 20   2,294     8,252   
3         169           6macb      Gold 1  15 /8/ 13   8,508     9,298   
4         169         TravyPG      Gold 1  14 /11/ 1  17,027    45,415   
...       ...             ...         ...        ...     ...       ...   
2018        1          Lephsh              15 /12/ 6  12,987     4,750   
2019        1     Paxton5Star               12 /9/ 7   6,046    10,136   
2020        1    ClassyRoboto              10 /13/ 0   7,388    12,817   
2021        1   TheAskaLottle              17 /10/ 0   8,140    16,907   
2022        1     Lildumdum12               16 /9/ 0  10,329     5,775   

     Healing                                      Heroes Played  \
0          0  Squirrel Girl / P